In [1]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import get_kraken_orderbook, get_omnipool_data
from hydradx.model.amm.centralized_market import CentralizedMarket
import hydradx.model.amm.arbitrage_agent as arb
import hydradx.model.amm.arbitrage_agent_new as arb2

from pprint import pprint

import json


In [2]:
with open('config/arbconfig.txt', 'r') as json_file:
    arb_list = json.load(json_file)
    
for d in arb_list:
    d['tkns'] = tuple(d['tkns'])
    d['tkn_ids'] = tuple(d['tkn_ids'])
    d['order_book'] = tuple(d['order_book'])

ob_objs = {}
order_book_asset_list = []

for arb_cfg in arb_list:
    tkn_pair = arb_cfg['order_book']
    if tkn_pair not in ob_objs:
        order_book_url = 'https://api.kraken.com/0/public/Depth?pair=' + tkn_pair[0] + tkn_pair[1]
        ob_objs[tkn_pair] = get_kraken_orderbook(tkn_pair, order_book_url)
        for tkn in tkn_pair:
            if tkn not in order_book_asset_list:
                order_book_asset_list.append(tkn)


In [3]:
RPC="wss://hydradx-rpc.dwellir.com"
N = 24
            
asset_list, asset_map, tokens, fees = get_omnipool_data(RPC, N)

pprint(tokens)


{'ASTR': {'LRNA': 9270.99688691519, 'liquidity': 4558461.91251756},
 'BNC': {'LRNA': 6527.933101450308, 'liquidity': 738947.6282724382},
 'CFG': {'LRNA': 9419.75435990446, 'liquidity': 592573.4619929172},
 'DAI': {'LRNA': 13000.817024566328, 'liquidity': 335561.52693695767},
 'DAI001': {'LRNA': 31429.245331500555, 'liquidity': 810044.2687278098},
 'DOT': {'LRNA': 71806.66123099784, 'liquidity': 420096.5614791631},
 'GLMR': {'LRNA': 5283.431188308908, 'liquidity': 673041.463199017},
 'HDX': {'LRNA': 19850.974136318495, 'liquidity': 115758722.44376807},
 'USDT': {'LRNA': 85706.77128217087, 'liquidity': 2211946.625641},
 'WBTC': {'LRNA': 50.601798806111, 'liquidity': 0.04374934},
 'WBTC001': {'LRNA': 41666.10085543079, 'liquidity': 31.19146557},
 'WETH': {'LRNA': 21426.4460746838, 'liquidity': 309.21785254735175},
 'WETH001': {'LRNA': 71262.38454994914, 'liquidity': 1023.6153847544036},
 'ZTG': {'LRNA': 4578.375407796498, 'liquidity': 3106881.31739501},
 'iBTC': {'LRNA': 34785.41880802545

In [4]:


cex_fee = 0.0016
buffer = 0.0010

lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list}
asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list}

# lrna_fee = 0.0
# asset_fee = 0.0
# cex_fee = 0.0
# buffer = 0.0


cex = CentralizedMarket(
    order_book=ob_objs,
    asset_list=order_book_asset_list,
    trade_fee=cex_fee
)

op_state = OmnipoolState(
    tokens=tokens,
    lrna_fee = lrna_fee,
    asset_fee = asset_fee,
    preferred_stablecoin='USDT',
)

init_state = op_state.copy()


In [5]:
order_book_map = {}
for i in range(len(arb_list)):
    base_id, quote_id = arb_list[i]['tkn_ids']
    orderbook_tkn_pair = arb_list[i]['order_book']
    if base_id in asset_map and quote_id in asset_map:
        tkn_pair = (asset_map[base_id], asset_map[quote_id])
        order_book_map[tkn_pair] = orderbook_tkn_pair 
    
pprint(order_book_map)

{('ASTR', 'DAI'): ('ASTR', 'USD'),
 ('ASTR', 'DAI001'): ('ASTR', 'USD'),
 ('ASTR', 'USDT'): ('ASTR', 'USD'),
 ('BNC', 'DAI'): ('BNC', 'USD'),
 ('BNC', 'DAI001'): ('BNC', 'USD'),
 ('BNC', 'USDT'): ('BNC', 'USD'),
 ('CFG', 'DAI'): ('CFG', 'USD'),
 ('CFG', 'DAI001'): ('CFG', 'USD'),
 ('CFG', 'USDT'): ('CFG', 'USD'),
 ('DOT', 'DAI'): ('DOT', 'USD'),
 ('DOT', 'DAI001'): ('DOT', 'USD'),
 ('DOT', 'USDT'): ('DOT', 'USD'),
 ('DOT', 'WBTC001'): ('DOT', 'XBT'),
 ('DOT', 'WETH'): ('DOT', 'ETH'),
 ('DOT', 'WETH001'): ('DOT', 'ETH'),
 ('DOT', 'iBTC'): ('DOT', 'XBT'),
 ('GLMR', 'DAI'): ('GLMR', 'USD'),
 ('GLMR', 'DAI001'): ('GLMR', 'USD'),
 ('GLMR', 'USDT'): ('GLMR', 'USD'),
 ('HDX', 'DAI'): ('HDX', 'USD'),
 ('HDX', 'DAI001'): ('HDX', 'USD'),
 ('HDX', 'USDT'): ('HDX', 'USD'),
 ('WBTC001', 'DAI'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'DAI001'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'USDT'): ('XXBT', 'ZUSD'),
 ('WETH', 'DAI'): ('XETH', 'ZUSD'),
 ('WETH', 'DAI001'): ('XETH', 'ZUSD'),
 ('WETH', 'USDT'): ('XETH', 'ZUS

In [6]:
cex1 = cex.copy()
op_state1 = op_state.copy()
all_swaps1 = arb.get_arb_swaps(op_state1, cex1, order_book_map, buffer=buffer)
tot_swaps = 0
for tkn_pair in all_swaps1:
    if all_swaps1[tkn_pair]:
        print(tkn_pair)
        print(all_swaps1[tkn_pair])
        tot_swaps += len(all_swaps1[tkn_pair])
print(tot_swaps)

cex2 = cex.copy()
op_state2 = op_state.copy()
all_swaps2 = arb2.get_arb_swaps(op_state2, cex2, order_book_map, buffer=buffer)
print("strategy 2")
pprint(all_swaps2)
pprint(len(all_swaps2))

('HDX', 'USDT')
[{'dex': {'trade': 'sell', 'buy_asset': 'USDT', 'sell_asset': 'HDX', 'price': 0.0044, 'amount': 11144.287276611329, 'min_buy': 49.14359470736981}, 'cex': {'trade': 'buy', 'buy_asset': 'HDX', 'sell_asset': 'USD', 'price': 0.0044022, 'amount': 11144.287276611329}}]
('HDX', 'DAI')
[{'dex': {'trade': 'sell', 'buy_asset': 'DAI', 'sell_asset': 'HDX', 'price': 0.0044, 'amount': 3239.6293021698, 'min_buy': 14.285292645275593}, 'cex': {'trade': 'buy', 'buy_asset': 'HDX', 'sell_asset': 'USD', 'price': 0.0044022, 'amount': 3239.6293021698}}]
('WETH', 'USDT')
[{'dex': {'trade': 'buy', 'buy_asset': 'WETH', 'sell_asset': 'USDT', 'price': 1801.91, 'amount': 0.012, 'max_sell': -27.657532707095143}, 'cex': {'trade': 'sell', 'buy_asset': 'ZUSD', 'sell_asset': 'XETH', 'price': 1801.0090450000002, 'amount': 0.012}}, {'dex': {'trade': 'buy', 'buy_asset': 'WETH', 'sell_asset': 'USDT', 'price': 1801.35, 'amount': 0.19758057403564452, 'max_sell': 327.2224709542393}, 'cex': {'trade': 'sell', 'b

In [7]:
state1 = init_state.copy()
state2 = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list + order_book_asset_list}
init_agent = Agent(holdings=holdings, unique_id='bot')
agent1 = init_agent.copy()
agent2 = init_agent.copy()
cex1 = cex.copy()
cex2 = cex.copy()

arb.execute_arb(state1, cex1, agent1, all_swaps1)
arb2.execute_arb(state2, cex2, agent2, all_swaps2)

asset_map = {'WETH': 'ETH',
             'XETH': 'ETH',
             'XXBT': 'BTC',
             'WBTC': 'BTC',
             'ZUSD': 'USD',
             'USDT': 'USD',
             'USDC': 'USD',
             'DAI': 'USD',
             'USDT001': 'USD',
             'DAI001': 'USD',
             'WETH001': 'ETH',
             'WBTC001': 'BTC',
             'iBTC': 'BTC',
             'XBT': 'BTC',}

profit1 = arb.calculate_profit(init_agent, agent1, asset_map)
profit2 = arb2.calculate_profit(init_agent, agent2, asset_map)

asset_map_omnipool = {
    'ETH': 'WETH',
    'XETH': 'WETH',
    'XXBT': 'WBTC',
    'BTC': 'WBTC',
    'ZUSD': 'USDT',
    'USD': 'USDT',
    'USDC': 'USDT',
    'DAI': 'USDT',
    'USDT001': 'USDT',
    'DAI001': 'USDT',
    'WETH001': 'WETH',
    'WBTC001': 'WBTC',
    'iBTC': 'WBTC',
    'XBT': 'WBTC',
}


profit1_omnipool = arb.calculate_profit(init_agent, agent1, asset_map_omnipool)
profit2_omnipool = arb2.calculate_profit(init_agent, agent2, asset_map_omnipool)

print("strategy 1 profit")
print(profit1)
print("strategy 1 profit at Omnipool spot")
print(sum([profit1_omnipool[asset] * state1.price(state1, asset, 'USDT') for asset in profit1_omnipool]))
print("strategy 2 profit")
print(profit2)
print("strategy 2 profit at Omnipool spot")
print(sum([profit2_omnipool[asset] * state2.price(state2, asset, 'USDT') for asset in profit2_omnipool]))
            

strategy 1 profit
{'HDX': 0.0, 'USD': 10.667392492294312, 'BTC': 0.00011169910430908203, 'ETH': 0.0, 'DOT': 0.0, 'ASTR': 0, 'ZTG': 0, 'CFG': 0, 'BNC': 0, 'vDOT': 0, 'GLMR': 0}
strategy 1 profit at Omnipool spot
13.996375439324042
strategy 2 profit
{'HDX': 0.0, 'USD': 0.06309187412261963, 'BTC': 0.000619053840637207, 'ETH': 0.0, 'DOT': 0.0, 'ASTR': 0, 'ZTG': 0, 'CFG': 0, 'BNC': 0, 'vDOT': 0, 'GLMR': 0}
strategy 2 profit at Omnipool spot
18.541887330297527
